In [ ]:
PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/Workspace/'
import sys
sys.path.insert(1, PROJECT_PATH + "Local-Approximation-Models-develop")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from sklearn.decomposition import PCA
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import math
from lam.utils import *

In [ ]:
def sphereFit(spX,spY,spZ):
    '''
    spX, spY, spZ: np.array([lenght])
    
    return:
    the radius and coordinates of the center of the ball, which was constructed from points by the method of least squares
    '''
    #   Assemble the A matrix
    spX = np.array(spX)
    spY = np.array(spY)
    spZ = np.array(spZ)
    A = np.zeros((len(spX),4))
    A[:,0] = spX*2
    A[:,1] = spY*2
    A[:,2] = spZ*2
    A[:,3] = 1

    #   Assemble the f matrix
    f = np.zeros((len(spX),1))
    f[:,0] = (spX*spX) + (spY*spY) + (spZ*spZ)
    C, residules, rank, singval = np.linalg.lstsq(A,f)

    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = math.sqrt(t)

    return radius, C[0], C[1], C[2]
    
def plot3d(trajectory, ind, sphere = False, projection = False):
  '''
  trajectory: np.array([lenght, N])
  ind: np.array(3) indexes of the 3 columns in which the rows to build are located
  sphere = True if you need to build a sphere by approximating the points with least squares
  projection = True if you need to project points on a sphere
  '''
  x = trajectory[:, ind[0]]
  y = trajectory[:, ind[1]]
  z = trajectory[:, ind[2]]
  if projection == True:
    s = np.sqrt(x**2 + y**2 + z**2)
    x = x/s
    y = y/s
    z = z/s

  fig = go.Figure(data=go.Scatter3d(
      x=x, y=y, z=z,
      marker=dict(
          size=3,
          color='red',
      ),
      line=dict(
          color='red',
          width=1.5
      )
  ))

  if sphere == True:
    if projection == True:
      u = np.linspace(0, 2 * np.pi, 100)
      v = np.linspace(0, np.pi, 100)
      x1 = 0.99 * np.outer(np.cos(u), np.sin(v))
      y1 = 0.99 * np.outer(np.sin(u), np.sin(v))
      z1 = 0.99 * np.outer(np.ones(np.size(u)), np.cos(v))
      x0 = y0 = z0 = 0
    else:
      r, x0, y0, z0 = sphereFit(x, y, z)
      u, v = np.mgrid[0:2*np.pi:20j, 0:np.pi:10j]
      x1=np.cos(u)*np.sin(v)*r
      y1=np.sin(u)*np.sin(v)*r
      z1=np.cos(v)*r
      x1 = x1 + x0
      y1 = y1 + y0
      z1 = z1 + z0
    fig.add_trace(go.Surface(x=x1, y=y1, z=z1, opacity = 0.4, surfacecolor = (x1-x0)**2+(y1-y0)**2+(z1-z0)**2 ))
  fig.show()

def comparison_plot3d(ind, trajectory1, trajectory2, trajectory3 = None):
  '''
  trajectory: np.array([lenght, N]) trajectories to be plotted in a single drawing
  ind: np.array(3) indexes of the 3 columns in which the rows to build are located
  
  '''
  x1 = trajectory1[:, ind[0]]
  y1 = trajectory1[:, ind[1]]
  z1 = trajectory1[:, ind[2]]
  x2 = trajectory2[:, ind[0]]
  y2 = trajectory2[:, ind[1]]
  z2 = trajectory2[:, ind[2]]
  fig = go.Figure(data=go.Scatter3d(
      x=x1, y=y1, z=z1,
      marker=dict(
          size=3,
          color='blue',
      ),
      line=dict(
          color='blue',
          width=1.5
      )
  ))
  fig.add_trace(go.Scatter3d(
      x=x2, y=y2, z=z2,
      marker=dict(
          size=3,
          color='red',
      ),
      line=dict(
          color='red',
          width=1.5
      )
  ))
  if trajectory3 is not None:
    x3 = trajectory3[:, ind[0]]
    y3 = trajectory3[:, ind[1]]
    z3 = trajectory3[:, ind[2]]
    fig.add_trace(go.Scatter3d(
      x=x3, y=y3, z=z3,
      marker=dict(
          size=3,
          color='green',
      ),
      line=dict(
          color='green',
          width=1.5
      )
    ))
  fig.show()

def to_phase_trajectory_multi(data, l):
  '''
  calculates its phase path for each column
  data: np.array([lenght, N]) time series
  l: int dimension of the space of phase trajectories
  return np.array([N, lenght, l])
  '''
  phases = np.zeros(data.shape[1], data.shape[0], l)
  for i, x in enumerate(data.T):
    phases[i] = to_phase_trajectory(x, l)
  return phases

def get_spectrum(series, l, n_comp = 2):
  '''
  projects phase trajectory of time series into a space of dimension n_comp
  series: np.array([lenght]) time series
  l: int dimension of the space of phase trajectories
  n_comp: int dimension projection phase trajectory
  return np.array([lenght, n_comp])
  '''
  phase = to_phase_trajectory(series, l)

  model = PCA(n_components = n_comp)
  projection = model.fit_transform(phase)
  basis = model.components_
  print('Explained variation'
        ' for {} principal components: {}'.format(n_comp,
                                                  model.explained_variance_ratio_))
  print('Cumulative explained variation'
        'for {} principal components: {}\n'.format(n_comp,
                                                  np.sum(model.explained_variance_ratio_)))
  return projection

def get_spectrum_multi(data, l, n_comp = 2):
  '''
  projects concat phase trajectories of time series into a space of dimension n_comp
  data: np.array([lenght, N]) time series
  l: int dimension of the space of phase trajectories
  n_comp: int dimension projection phase trajectory
  return np.array([lenght, n_comp])
  '''
  spectrums = np.zeros([data.shape[0] - l, data.shape[1]*n_comp])
  for i, x in enumerate(data.T):
    spectrums[:, n_comp*i : n_comp*i+n_comp] = get_spectrum(x, l, n_comp = n_comp)
  model = PCA(n_components = n_comp)
  spectrums = model.fit_transform(spectrums)
  return spectrums

def plot2d(x, y):
  plt.figure(figsize=(6, 6))
  plt.plot(x, y)
  plt.show()